In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from core import CompanyProduct

target = CompanyProduct("Rad AI", "Omni")

In [3]:
from glassdoor.search import find_review_url

url = find_review_url(target, debug=True)

['https://www.glassdoor.com/Reviews/Rad-AI-Reviews-E3543079.htm', 'https://www.glassdoor.com/Reviews/Rad-AI-team-Reviews-EI_IE3543079.0,6_KH7,11.htm', 'https://www.glassdoor.com/Reviews/Rad-AI-Berkeley-Reviews-EI_IE3543079.0,6_IL.7,15_IC1147330.htm', 'https://www.glassdoor.com/Reviews/Rad-AI-Head-of-Research-Reviews-EI_IE3543079.0,6_KO7,23.htm']


In [4]:
from glassdoor.scraper import scrape_reviews, GlassdoorReview

glassdoor_results = await scrape_reviews(url, max_pages=3)
reviews = GlassdoorReview.parse_reviews(glassdoor_results)


2024-07-29 18:01:13.614 | INFO     | glassdoor.scraper:scrape_reviews:110 - scraping reviews from https://www.glassdoor.com/Reviews/Rad-AI-Reviews-E3543079.htm
2024-07-29 18:01:13.959 | INFO     | glassdoor.scraper:scrape_reviews:118 - scraped first page of reviews of https://www.glassdoor.com/Reviews/Rad-AI-Reviews-E3543079.htm, scraping remaining 1 pages
2024-07-29 18:01:15.469 | INFO     | glassdoor.scraper:scrape_reviews:128 - scraped 13 reviews from https://www.glassdoor.com/Reviews/Rad-AI-Reviews-E3543079.htm in 2 pages


In [5]:
from glassdoor.summarizer import summarize

summary = summarize(target, reviews)
print(summary.content)

The prompt context has 12,927 characters in 13 reviews
# Summary of Glassdoor Reviews for Rad AI

## Key Aspects

### Leadership
**Pros:**
- Some reviews highlight strong leadership from co-founders, particularly Jeff Chang, who is described as caring and visionary.
- Open and supportive leadership is noted in several positive reviews.

**Cons:**
- Many reviews criticize the current leadership for being egotistical, inexperienced, and incompetent.
- There are reports of a toxic culture stemming from dysfunctional leadership and a lack of trust in employees.

**Quotations:**
- "Egotistical, inexperienced, incompetent leadership" (Senior Software Engineer on 2021-04-13)
- "Co-Founders are wonderful... The next level of leadership is where the problem lies." (Anonymous on 2023-02-17)

### Compensation and Benefits
**Pros:**
- Good pay and decent benefits are mentioned positively in several reviews.
- Equity is noted as likely to be valuable.

**Cons:**
- Some reviews indicate that compens